In [10]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')
edenai_api_key = os.getenv('EDENAI_API_KEY')
huggingface_api_key = os.getenv('HF_TOKEN')
openai.api_key  = os.getenv('OPENAI_API_KEY')

# print(openai.api_key[0:2])
# print(edenai_api_key[0:2])
# print(huggingface_api_key[0:2])

Als je de print commando's uitvoert, zou je de key(s) die je wilt gebruiken moeten zien, als je `None` ziet staan is er iets mis.

De key van EdenAI is een Bearer token, daar kennen jullie alles van uit de Webservices cursus. Rechtstreeks de EdenAI API aanspreken doe je met een POST request naar de juiste url.
EdenAI heeft véél endpoints, een ChatGPT request zou er als volgt uitzien:

In [42]:
from enum import Enum

from langchain.llms import EdenAI, OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.embeddings.edenai import EdenAiEmbeddings
from langchain.embeddings import OpenAIEmbeddings

class API(Enum):
    OPEN_AI = 1
    GPT4All = 2
    EDEN_AI = 3
    HUGGINGFACE = 4
    

def get_llm(which_model=API.OPEN_AI, temperature = 0.0):
    if which_model == API.OPEN_AI:
        # OpenAI heeft ook een ChatModel, maar dat is niet makkelijk transparent te gebruiken als je ook EdenAI wil gebruiken in dezelfde requests. Als je enkel met OpenAI wenst te werken is dit zeker een goede optie. Je zal dan wel het output formaat moeten aanpassen.
        # return ChatOpenAI(temperature=temperature, model="gpt-3.5-turbo")
        return OpenAI(temperature=temperature)
    elif which_model == API.EDEN_AI:
        return EdenAI(edenai_api_key=edenai_api_key,provider="openai", model="gpt-3.5-turbo-instruct", temperature=temperature, max_tokens=250)
    # elif which_model == API.HUGGINGFACE:
    #     return InferenceClient("NousResearch/Hermes-3-Llama-3.1-8B")
        
def get_embedding(which_model=API.OPEN_AI):
    if which_model == API.OPEN_AI:
        return OpenAIEmbeddings()
    elif which_model == API.EDEN_AI:
        return EdenAiEmbeddings(edenai_api_key=edenai_api_key,provider="openai")
    # elif which_modem == API.HUGGINGFACE:
    #     client = InferenceClient(sentence-transformers/all-MiniLM-L6-v2")
    #     hf_token = os.getenv('HF_TOKEN')
    #     api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
    #     headers = {"Authorization": f"Bearer {hf_token}"}
    #     response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})
    #     return response.json()

# model_id = "sentence-transformers/all-MiniLM-L6-v2"
# model_id = "sentence-transformers/all-MiniLM-L12-v2"
# model_id = "sentence-transformers/multi-qa-mpnet-base-dot-v1"


In [48]:
#!pip install -U langchain-openai

In [49]:
from langchain_openai import OpenAI

In [50]:
prefered_model = API.OPEN_AI
# prefered_model = API.HUGGINGFACE
# llm = get_llm(API.EDEN_AI, 0.0)
# creative_llm = get_llm(API.EDEN_AI, 0.9)
llm = get_llm(prefered_model, 0.0)
creative_llm = get_llm(prefered_model, 0.9)
embeddings = get_embedding(prefered_model)


# Chat with your data

![langchain](img/langchain.jpg)

(image credit: langchain.com)


## Document Loading

Extra data kan in allerlei formaten voorkomen, PDF, JSON, tekst, ... en kan zowel gestructureerd of ongestructureerd zijn.

Om dit te illustreren starten we met een PDF van "The Little Book on Deep Learning" (https://fleuret.org/public/lbdl.pdf), vooral omdat deze een creative commons license heeft.

In [ ]:
# !pip install -qU pypdf
# !pip install langchain_community

In [40]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/lbdl.pdf")
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [14]:
print(pages[15].page_content[:500])
print(pages[15].metadata)

1.3 Under and overfitting
A key element is the interplay between the capac-
ity of the model, that is its flexibility and ability
to fit diverse data, and the amount and quality
of the training data. When the capacity is insuf-
ficient, the model cannot fit the data, resulting
in a high error during training. This is referred
to as underfitting.
On the contrary, when the amount of data is in-
sufficient, as illustrated in Figure 1.2, the model
will often learn characteristics specific to the
tra
{'source': 'data/lbdl.pdf', 'page': 15}


### Youtube

Maar je kan ook youtube video's als input nemen, we combineren de audio loader van youtube, and de OpenAI whisper parser, die samen een youtube video omzetten in een stuk tekst dat als bron kan dienen.
(om dit te doen werken heb je ook ffmpeg nodig, MacOS/Linux: `brew/apt-get/... install ffmpeg`, Windows: downloaden en prutsen)



In [4]:
# yt_dlp heeft ffmpeg nodig; dus we zorgen dat het in het standaard path gevonden wordt
# onderstaande paden zijn typisch voor MacOS homebrew, je zal ze waarschijnlijk moeten aanpassen naargelang je OS en waar je ffmpeg geïnstalleerd hebt
import sys
sys.path.append('/opt/homebrew/bin/ffmpeg')
sys.path.append('/opt/homebrew/bin/ffprobe')

In [5]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser # enkel OpenAI
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [30]:

# from yt_dlp.postprocessor import FFmpegPostProcessor
# FFmpegPostProcessor._ffmpeg_location.set('/opt/homebrew/bin/')

<Token var=<ContextVar name='ffmpeg_location' default=None at 0x177e5f510> at 0x28d1e3640>

In [7]:
# OPGELET
#
# dit deel uitvoeren duurt snel een aantal minuten; audio transcriben is ook relatief duur $0.006/min of dus $0.36 per uur
# geen extreme bedragen, maar voer dit stuk code ook niet te vaak uit

# Transformer Neural Networks van CodeEmporium
url="https://www.youtube.com/watch?v=TQQlZhbC5ps"

save_dir="data/youtube/"

loader = GenericLoader(
    YoutubeAudioLoader([url],save_dir),
    OpenAIWhisperParser()
)

docs = loader.load()
docs[0].page_content[0:500]

[youtube] Extracting URL: https://www.youtube.com/watch?v=TQQlZhbC5ps
[youtube] TQQlZhbC5ps: Downloading webpage
[youtube] TQQlZhbC5ps: Downloading ios player API JSON
[youtube] TQQlZhbC5ps: Downloading android player API JSON
[youtube] TQQlZhbC5ps: Downloading m3u8 information
[info] TQQlZhbC5ps: Downloading 1 format(s): 140
[download] Destination: data/youtube//Transformer Neural Networks - EXPLAINED! (Attention is all you need).m4a
[download] 100% of   12.11MiB in 00:00:13 at 908.91KiB/s 
[FixupM4a] Correcting container of "data/youtube//Transformer Neural Networks - EXPLAINED! (Attention is all you need).m4a"
[ExtractAudio] Not converting audio data/youtube//Transformer Neural Networks - EXPLAINED! (Attention is all you need).m4a; file is already in target format m4a
Transcribing part 1!
Transcribing part 1!
Transcribing part 2!
Transcribing part 3!
Transcribing part 4!


'Recurrent neural nets, they are feed-forward neural networks rolled out over time. As such, they deal with sequence data, where the input has some defined ordering. This gives rise to several types of architectures. The first is vector-to-sequence models. These neural nets take in a fixed-size vector as input, and it outputs a sequence of any length. In image captioning, for example, the input can be a vector representation of an image, and the output, sequence, is a sentence that describes the '

### URL

Je kan ook gewoon alle content van een url inladen, om dan met de data van een website te 'chatten'. De standaard `WebBaseLoader` kan HTML inlezen en beschikbaar maken, dit werkt wel enkel voor sites die niet JavaScript 'heavy' zijn. Om sites die dynamisch opgebouwd worden te kunnen laden heb je een headless browser nodig. Selenium, te gebruiken via `SeleniumURLLoader` is een optie. (hoewel die dingen niet altijd even perfect werken)   

In [45]:

from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import SeleniumURLLoader
loader = WebBaseLoader("https://raw.githubusercontent.com/HOGENT-Web/csharp/main/chapters/03/slides/presentation.md")

docs = loader.load()

print(docs[0].page_content[:500])

class: dark middle

# Enterprise Web Development C#
> Chapter 3 - Solving The Problem Domain

---
### Chapter 3 - Solving The Problem Domain
# Table of contents

- [The Visual Studio Solution](#vs-solution)
- [The sample application](#sample-application)
- [Classes](#classes)
- [Associations & collections](#associations)
- [Inheritance](#inheritance)
- [Polymorphism](#polymorphism)
- [Abstract class](#abstract-class)
- [Interface](#interface)
- [Static members](#static-members)
- [Domain Driven 


## Document Splitting

We willen de tekst in stukken splitsen, maar dat is veel subtieler dan je zou denken. Je wilt de tekst semantisch splitsen, de stukken zullen later een vector encodering krijgen en gebruikt worden om vragen te beantwoorden. Simpel gezegd wil je bijvoorbeeld niet dat een zin half in één stuk zit, en half in een ander.

Er zijn een aantal splitters voorzien in [langchain](https://python.langchain.com/docs/modules/data_connection/document_transformers/#text-splitters): `CharacterTextSplitter`, `MarkdownHeaderTextSplitter`, `TokenTextSplitter`, `RecursiveCharacterTextSplitter` enz.

In [15]:

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# de grootte van een stuk
chunk_size = 26
# overlap tussen stukken
chunk_overlap = 4

c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)


In [16]:
# past net in één stuk, dus de splitter doet sowieso niets
text1 = 'abcdefghijklmnopqrstuvwxyz'
c_splitter.split_text(text1)

['abcdefghijklmnopqrstuvwxyz']

In [17]:
# we voegen spaties toe om de tekst dubbel zo lang te maken
text1 = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'
c_splitter.split_text(text1)

['a b c d e f g h i j k l m n o p q r s t u v w x y z']

De `CharacterTextSplitter` doet ogenschijnlijk niets, dit komt omdat er gesplitst wordt op basis van een specifiek character (standaard een newline), en dus zal een text zonder newlines niet gesplitst worden 


In [18]:
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator=' ' # default, hier expliciet meegegeven
)
text1 = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'
c_splitter.split_text(text1)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

In de praktijk is deze splitter dus niet zo handig, je wilt liefst zo semantisch mogelijk splitsen (per paragraaf of zin) en pas als die paragrafen / zinnen te lang worden, op basis van een ander character.

Daartoe dient de `RecursiveCharacterTextSplitter`, i.p.v. één separator character te gebruiken, geef je een lijst van separators mee, er zal eerst gesplitst worden volgens het eerste character, als de stukken dan nog te lang zijn opnieuw via de tweede, enz.
Tot er geen separators meer over zijn of elk stuk klein genoeg geworden is 

In [19]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", " ", ""] # default, dus eerst per paragraaf, dan per zin, dan per woord, dan gewoon op character
)


In [20]:
# als enkel een spatie als separator voorkomt, gedraagt het zich dus identiek aan de `CharacterTextSplitter`
text1 = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'
r_splitter.split_text(text1)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

In [21]:
# we illustreren met een realistisch stuk tekst
text2 = pages[15].page_content[:700]
print(text2)

1.3 Under and overfitting
A key element is the interplay between the capac-
ity of the model, that is its flexibility and ability
to fit diverse data, and the amount and quality
of the training data. When the capacity is insuf-
ficient, the model cannot fit the data, resulting
in a high error during training. This is referred
to as underfitting.
On the contrary, when the amount of data is in-
sufficient, as illustrated in Figure 1.2, the model
will often learn characteristics specific to the
training examples, resulting in excellent perfor-
mance during training, at the cost of a worse
Figure 1.2: If the amount of training data (black dots)
is small compared to the capacity of the model, the


In [22]:
chunk_size = 150
chunk_overlap = 0
c_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, separator=' ')
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

c_splitter.split_text(text2)

['1.3 Under and overfitting\nA key element is the interplay between the capac-\nity of the model, that is its flexibility and ability\nto fit diverse data,',
 'and the amount and quality\nof the training data. When the capacity is insuf-\nficient, the model cannot fit the data, resulting\nin a high error during',
 'training. This is referred\nto as underfitting.\nOn the contrary, when the amount of data is in-\nsufficient, as illustrated in Figure 1.2, the',
 'model\nwill often learn characteristics specific to the\ntraining examples, resulting in excellent perfor-\nmance during training, at the cost of a',
 'worse\nFigure 1.2: If the amount of training data (black dots)\nis small compared to the capacity of the model, the']

In [23]:
# soms iets kleinere stukken nemen, maar gesplitst op basis van een newline, geef betere resultaten
r_splitter.split_text(text2)

['1.3 Under and overfitting\nA key element is the interplay between the capac-\nity of the model, that is its flexibility and ability',
 'to fit diverse data, and the amount and quality\nof the training data. When the capacity is insuf-\nficient, the model cannot fit the data, resulting',
 'in a high error during training. This is referred\nto as underfitting.\nOn the contrary, when the amount of data is in-',
 'sufficient, as illustrated in Figure 1.2, the model\nwill often learn characteristics specific to the',
 'training examples, resulting in excellent perfor-\nmance during training, at the cost of a worse',
 'Figure 1.2: If the amount of training data (black dots)\nis small compared to the capacity of the model, the']

In [24]:
# nu toegepast op het volledig document
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
split_pages = r_splitter.split_documents(pages)


In [25]:
print(len(split_pages))
print(len(pages))

215
168


Een andere manier is om te splitsen op basis van tokens. Dit is belangrijk als we met LLM's werken, die een context hebben die beperkt is tot een aantal tokens

In [26]:
from langchain.text_splitter import TokenTextSplitter
t_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)

In [38]:
text1 = "Dit is belangrijk"
t_splitter.split_text(text1)

['D', 'it', ' is', ' bel', 'ang', 'ri', 'j', 'k']

Voor markdown files gebruiken we best de `MarkdownHeaderTextSplitter`, zoals de naam suggereert worden markdown files gesplitst op basis van de headers, en de informatie uit die headers komt dan in de metadata terecht.

In [32]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.document_loaders import WebBaseLoader

m_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("#", "header 1"), ("##", "header 2")])

# loader = WebBaseLoader("https://raw.githubusercontent.com/HOGENT-Web/csharp/main/chapters/03/slides/presentation.md")
loader = WebBaseLoader("https://github.com/VeerleDepestele/Trends_in_AI/blob/master/les3_0_quantization.md")
markdown_doc = loader.load()

# print(markdown_doc[0].page_content[:500])

m_split_text = m_splitter.split_text(markdown_doc[0].page_content)
# print (m_split_text[0])
# print (m_split_text[0].metadata)


### vectorstores

De volgende stap is al deze chunks in een vectorstore op te slaan, zodat we snel en makkelijk 'gelijkaardige' content kunnen terugvinden (die we dan samen met onze vraag naar een LLM doorsturen, maar dat komt later)

We dienen eerste vector embeddings te creëren, vector voorstellingen van onze stukken tekst, hiervoor gebruiken we OpenAI

In [64]:
# klein voorbeeld om te tonen dat gelijkaardige tekst tot gelijkaardige embeddings leidt. 

embedding1 = embeddings.embed_query("De student dronk iets te veel op de cantus")
embedding2 = embeddings.embed_query("De student drinkt graag bier op feestjes")
embedding3 = embeddings.embed_query("A completely different piece of text about a man who's working very hard on course material")

# print(embedding1)
# print(embedding2)
# print(embedding3)

# het dot product geeft aan in hoeverre twee vectoren in dezelfde richting 'wijzen'
# m.a.w. hoe groter het dot product, hoe gelijkaardiger twee vectoren zijn
import numpy as np

print(np.dot(embedding1, embedding2))
print(np.dot(embedding1, embedding3))
print(np.dot(embedding2, embedding3))


0.903618173120376
0.7657950951685688
0.7552190470689308


### Chroma

Chroma is een vectorstore die volledig in het geheugen draait, ideaal om snel wat code te runnen (en te demonstreren hier), voor grotere toepassingen bestaan er veel gehoste oplossing ook, langchain heeft bindings voor de meest courant gebruikte.

In [51]:
from inspect import signature

t = signature(embeddings.embed_documents)
print (t)

(texts: 'List[str]', chunk_size: 'Optional[int]' = 0) -> 'List[List[float]]'


In [61]:
# !pip install chromadb
# !pip install -qU "langchain-chroma>=0.1.2"

In [62]:
from langchain_chroma import Chroma

chroma_dir = "data/chroma/"

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory=chroma_dir,  # Where to save data locally, remove if not necessary
)

In [63]:
from uuid import uuid4

from langchain_core.documents import Document

In [75]:
uuids = [str(uuid4()) for _ in range(len(split_pages))]

vector_store.add_documents(documents=split_pages, ids=uuids)

['48ecd4d6-8ca3-48a4-89f4-f32366da5620',
 '5aa1f3d4-7f23-421c-94fd-3897913c3bd6',
 'edf55386-4be9-406c-9ff2-a736bb15be41',
 '4247a9b9-216f-40d3-b7b7-ccf7c801aeba',
 'e8e228fd-228a-4400-afd7-db880367390a',
 '7b51eec8-e17e-4bd8-8395-d6e0723617cf',
 'dcad173a-b691-47b8-9c48-fc264cec7902',
 '61b4be47-2b3d-4cfc-afa8-d899a8888e0a',
 'e1515f1d-85d8-457f-9997-34d81b9d6843',
 '1da8819c-8133-4a76-b478-c0287ac6d551',
 'b134b77a-d4f0-427d-8e1b-2727cb98ec4d',
 'b5217b26-eff1-4e89-ad71-79cd6410dad1',
 '882b54fd-1ca7-4c16-a7ac-f6fb0396ca7e',
 'e02310df-4f6f-40d3-bfff-b47efaf7e08c',
 'b8a9c03f-a39f-4025-aa7f-b5e8a88b176e',
 'b9ea84b5-1b9c-4453-8e6e-e4bbca599a35',
 '5a4686f9-35de-4343-86ec-8837e93bb394',
 '316bfe5b-192b-4a5b-a344-fe9f6327adab',
 'f992f6ea-97d8-46e5-9640-0ad2a99fe49f',
 '8b680c9e-4e1b-4a31-8fa6-6a8520797865',
 '88b4d2f0-e0c9-4330-8783-57d5cd1a192d',
 '191b1ebb-edbb-4cde-922a-c6e9378f0fbd',
 '64edff38-72af-4ec8-8297-bfe920efcf5a',
 'a0de31cf-756a-4a2d-96ad-05a6ef4f55e9',
 '63ea9d03-aa0f-

In [ ]:
# vector_store.add_documents(documents=m_split_text, ids=uuids)

In [76]:
# print(chromadb._collection.count())
print(vector_store._collection.count())

216


In [77]:
chromadb = vector_store

In [78]:
question = "How does polymorphism work in C#?"

result = chromadb.similarity_search(question, k=3)

In [79]:
print(len(result))
print(result[0].page_content)

3
visualization, while real models take advantage
of representations in high dimensions, which, in
particular, facilitates the optimization by provid-
ing many degrees of freedom.
Empirical evidence accumulated over twenty
years demonstrates that state-of-the-art perfor-
mance across application domains necessitates
models with tens of layers, such as resid ualnet-
works (see § 5.2) or Trans form ers (see § 5.3).
Theoretical results show that, for a fixed com-
putational budget or number of parameters, in-
creasing the depth leads to a greater complexity
of the resulting mapping [Telgarsky, 2016].
47


### Retrieval

Maximum marginal relevance (MMR), als je gewoon de meest gelijkende resultaten zoekt, gebeurt het soms dat je een aantal resultaten terug krijgt die redundant zijn; die eigenlijk allemaal dezelfde inhoud terug geven. MMR kan hier helpen, het algoritme zal naast de relevantie van de resultaten, ook de 'diversiteit' in rekening nemen, en op basis van beide een nieuwe ranking maken. 






In [5]:

from langchain.vectorstores import Chroma

persist_directory = 'data/chroma/'

chromadb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

In [9]:
# result was het resultaat van een similarity_search
print(result)

[Document(page_content='> Polymorphism  \n---\n### Solving The Problem Domain', metadata={'header 1': 'Solving The Problem Domain'}), Document(page_content='* **Every class** in C# implicitly **inherits from `System.Object`**\n* No need to write this\n* You get these methods for free with this behaviour\n* `ToString()`: returns the class name\n* `Equals(Object)`: returns true\n* if two reference variables reference the same object or\n* if two value variables have the same value\n* `GetHashCode()`: used in hash-based collections (e.g. `Dictionary`)  \n---\n### Solving The Problem Domain  \n* **Superclass** contains shared properties and methods\n* **Subclass inherits all** `public` and `protected` members\n* `private` members are not accessible/inherited\n* **Subclass extends or specialises** the superclass\' behavior\n* **"Is a"** relation between sub- and superclass  \n---\n### Inheritance', metadata={'header 1': 'Inheritance'}), Document(page_content='C# has different types of colle

In [80]:
chromadb.max_marginal_relevance_search(question, k=3, fetch_k=5)

[Document(metadata={'page': 46, 'source': 'data/lbdl.pdf'}, page_content='visualization, while real models take advantage\nof representations in high dimensions, which, in\nparticular, facilitates the optimization by provid-\ning many degrees of freedom.\nEmpirical evidence accumulated over twenty\nyears demonstrates that state-of-the-art perfor-\nmance across application domains necessitates\nmodels with tens of layers, such as resid ualnet-\nworks (see § 5.2) or Trans form ers (see § 5.3).\nTheoretical results show that, for a fixed com-\nputational budget or number of parameters, in-\ncreasing the depth leads to a greater complexity\nof the resulting mapping [Telgarsky, 2016].\n47'),
 Document(metadata={'page': 137, 'source': 'data/lbdl.pdf'}, page_content='“because her puppy was sick, Jane was”.\nThis results in particular in the ability to solve\nfew-shot prediction, where only a handful of\ntraining examples are available, as illustrated\nin Figure 7.1. More surprisingly, when gi

SelfQuery, een ander algoritme dat je met langchain kan gebruiken is SelfQuery, het idee is dat je je vraag stelt in 'natuurlijke taal', en dat de LLM zichzelf gebruikt (vandaar de naam) om onderscheid te maken tussen delen van de vraag waarmee de metadata kan gefilterd worden, en de eigenlijk inhoud zelf. 

Als we kijken welke metadata er in het resultaat van onze similarity_search zit

In [10]:
for d in result:
    print (d.metadata)

{'header 1': 'Solving The Problem Domain'}
{'header 1': 'Inheritance'}
{'header 1': 'Implementations'}


In [13]:
from langchain.chains.query_constructor.base import AttributeInfo
metadata_info = [AttributeInfo(name="header 1", description="The subject of this part of the course", type="string")]

In [11]:
from langchain.retrievers.self_query.base import SelfQueryRetriever

self_retriever = SelfQueryRetriever.from_llm(llm, chromadb, "C# programming course", metadata_info, verbose=True)

In [12]:
docs = self_retriever.get_relevant_documents("When is polymorphism useful?")
print(docs)

[Document(page_content='> Polymorphism  \n---\n### Solving The Problem Domain', metadata={'header 1': 'Solving The Problem Domain'}), Document(page_content='Now implement the `SavingsAccount` class!  \n---\nname: polymorphism\nclass: dark middle', metadata={'header 1': 'Example'}), Document(page_content='> Domain Driven Design  \n---\n### Domain Driven Design', metadata={'header 1': 'Solving The Problem Domain'}), Document(page_content='> Inheritance  \n---\n### Solving The Problem Domain', metadata={'header 1': 'Solving The Problem Domain'})]


In [14]:
docs = self_retriever.get_relevant_documents("When is polymorphism useful, exclude examples")
print(docs)

[Document(page_content='> Polymorphism  \n---\n### Solving The Problem Domain', metadata={'header 1': 'Solving The Problem Domain'}), Document(page_content='> Domain Driven Design  \n---\n### Domain Driven Design', metadata={'header 1': 'Solving The Problem Domain'}), Document(page_content='> Inheritance  \n---\n### Solving The Problem Domain', metadata={'header 1': 'Solving The Problem Domain'}), Document(page_content='> Abstract class  \n---\n### Solving The Problem Domain', metadata={'header 1': 'Solving The Problem Domain'})]


### Vragen beantwoorden

We hebben gezien hoe we data kunnen inladen, hoe we documenten kunnen splitten en in een vectorstore krijgen, en hoe we dan een relevant document kunnen opvragen voor onze vragen.

De volgende stap is nu de LLM deze vraag te laten beantwoorden, met behulp van dit document

In [81]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm, retriever=chromadb.as_retriever())


In [82]:
result = qa_chain({"query" : "When is polymorphism useful?"})
print(result)

/tmp/ipykernel_275/1172359795.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query" : "When is polymorphism useful?"})


{'query': 'When is polymorphism useful?', 'result': " I don't know."}


In [83]:
result = qa_chain({"query": "Give an example of where polymorphism can be used?"})
print(result)

{'query': 'Give an example of where polymorphism can be used?', 'result': ' Polymorphism can be used in programming languages to create functions or methods that can operate on different types of data. For example, a function that calculates the area of a shape can be polymorphic, meaning it can work with different types of shapes such as circles, squares, or triangles. This allows for more flexible and reusable code.'}


In [84]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks to HOGENT course!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Run chain

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=chromadb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [85]:
result = qa_chain({"query": "Give an example of where polymorphism can be used?"})
print(result["result"])

 Polymorphism can be used in programming languages to create functions or objects that can operate on different types of data. For example, a function that can add two numbers together can also be used to concatenate two strings. Thanks to HOGENT course!


In [86]:
print(result["source_documents"][0])

page_content='“because her puppy was sick, Jane was”.
This results in particular in the ability to solve
few-shot prediction, where only a handful of
training examples are available, as illustrated
in Figure 7.1. More surprisingly, when given a
carefully crafted prompt, it can exhibit abilities
138' metadata={'page': 137, 'source': 'data/lbdl.pdf'}


#### map reduce

Als de documenten te uitgebreid zijn, zullen ze al snel groter zijn dan de beschikbare context voor LLM's. Een oplossing is om map reduce toe te passen, simpel gezegd zal je de documenten opsplitsen, de vraag naar elk sturen 'mappen', en dan de verschillende antwoorden 'reducen'.

Dit leidt snel tot vrij veel API calls dus we gaan dit hier niet demonstreren. Er zijn voorbeelden en uitleg te vinden op langchain als je dit nodig hebt.


### Chat

Om nu echt te kunnen chatten met de data ontbreekt er nog een deel van de puzzel, we moeten de eerder gegeven antwoorden ook kunnen meenemen in een volgende vraag. Zodat we extra verduidelijking kunnen krijgen, zoals we dat tegenwoordig gewend zijn van chatbots.


In [87]:
# wat we missen is een chat geheugen
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

qa_conversation = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=chromadb.as_retriever(),
    memory=memory
)


/tmp/ipykernel_275/3025019359.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [29]:
# opgelet! de key is nu 'question' en niet 'query'
result = qa_conversation({"question": "Give an example of where polymorphism can be used?"})
print(result)

{'question': 'Give an example of where polymorphism can be used?', 'chat_history': [HumanMessage(content='Give an example of where polymorphism can be used?'), AIMessage(content=' Polymorphism can be used when creating a class hierarchy, where a parent class is inherited by multiple child classes. For example, a `BankAccount` class can be inherited by a `SavingsAccount` class and a `CheckingAccount` class.'), HumanMessage(content='Give an example of where polymorphism can be used?'), AIMessage(content=' Polymorphism can be used to create a class hierarchy by having a parent class and then creating child classes that inherit the properties of the parent class.')], 'answer': ' Polymorphism can be used to create a class hierarchy by having a parent class and then creating child classes that inherit the properties of the parent class.'}


In [30]:
result = qa_conversation({"question": "What would an implementation of a SavingsAccount look like?"})
print(result)

{'question': 'What would an implementation of a SavingsAccount look like?', 'chat_history': [HumanMessage(content='Give an example of where polymorphism can be used?'), AIMessage(content=' Polymorphism can be used when creating a class hierarchy, where a parent class is inherited by multiple child classes. For example, a `BankAccount` class can be inherited by a `SavingsAccount` class and a `CheckingAccount` class.'), HumanMessage(content='Give an example of where polymorphism can be used?'), AIMessage(content=' Polymorphism can be used to create a class hierarchy by having a parent class and then creating child classes that inherit the properties of the parent class.'), HumanMessage(content='What would an implementation of a SavingsAccount look like?'), AIMessage(content=' Checking the type of the instance is possible with the `is` keyword: \n```\nBankAccount s = new SavingsAccount("123-123123-13", 0.1M)\nif (s is SavingsAccount)\n{\n// Do something useful\n}\n```')], 'answer': ' Chec